In [1]:
import pandas as pd
import requests
import ast
from datetime import datetime, timedelta
pd.options.display.max_columns = None

# Pair Programming ETL Transformación II

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [2]:
# Creamos la clase y el metodo constructor con la variable diccionario donde estarán los paises y sus coordenadas
class Extraccion:
        def __init__(self, dicc):
    
            self.dicc = dicc
        
        # Creamos un método para realizar la llamada a la API
        def llamada_API(self, producto):

            self.producto = producto
            #Iteramos por el diccionario en busca de nuestros valores
            lista_dataframes = []
            for k, v in self.dicc.items():
                lat = v[0]
                lon = v[1]
            # hacemos la llamada  a la API
                url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
                
                response = requests.get(url=url)
                codigo_estado = response.status_code
                razon_estado = response.reason
            # Ponemos esto para comprobar si todo está correcto.    
                if codigo_estado == 200:
                    print('La peticion de se ha realizado correctamente, se ha devuelto el código de estado:',codigo_estado,' y como razón del código de estado: ',razon_estado)
                elif codigo_estado == 402:
                    print('No se ha podido autorizar usuario, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
                elif codigo_estado == 404:
                    print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
                else:
                    print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', codigo_estado,' y como razón del código de estado: ',razon_estado)
                
            # convertimos los resultados en un dataframe:
                df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
            # Creamos columnas con el nombre del pais, la latitud y la longitud para poder hacer la union.
                df["country"] = k
                df["latitud"] = lat
                df["longitud"] = lon

                # Apendeamos el dataframe en una lista que hemos creado
                lista_dataframes.append(df)
    
            # juntamos todos los dataframes
            data = pd.concat(lista_dataframes, axis= 0, ignore_index= True)
            return data

        # Definimos una función para trabajar con las columnas que tienen una lista de diccionarios
        # Al llamar a la funcion hay que pasarle el nombre del dataframe y de la columna donde hay un diccionario
        def separar_lista_df(self,df,col):
            self.df = df
            self.col = col

            # Las listas estaban en formato string, por lo que lo convertimos para que sean listas y poder separarlas en columnas con pd.Series
            #df[col] = df[col].apply(ast.literal_eval)
            df_final = df[col].apply(pd.Series)
            # Nos devuelve un dataframe donde cada columna será uno de los diccionarios que están en la lista
            return df_final
    
        # Al desempaquetar hay que pasar el dataframe inicial donde se va a unir todo, y el datrafame que hemos creado al separar la lista
        def desempaquetar(self,datf,df_inicial):
            self.datf = datf
            self.df_inicial = df_inicial

            # Mediante un imput, le preguntamos el nombre del dataframe para saber si es wind o rh
            df_elegido = input("¿Cual es tu dataframe? Escoge entre wind y rh")
            # Si es wind, mediante otro input le preguntamos si es direction o speed
            if df_elegido == "wind":
                nombre = input('¿Cual tu parametro? Escoge entre direction y speed')
                for i in range(len(datf.columns)):
                    # Hacemos lo mismo para speed
                    nombre2= f'{df_elegido}_{nombre}_' + str(datf[i].apply(pd.Series)["layer"][0])
                    valores2 = list(datf[i].apply(pd.Series)[nombre])
                    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
                    df_inicial.insert(i, nombre2, valores2)
                return df_inicial
            # Si es rh, no nos pide nada más
            elif df_elegido == "rh": 
                for i in range(len(datf.columns)):
                    #Aplicamos el apply y extraemos el valor de la key "layer" y lo almacenamos en una variable que convertimos a string
                    nombre = "rh_" + str(datf[i].apply(pd.Series)["layer"][0])
                    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
                    valores = list(datf[i].apply(pd.Series)["rh"])
                    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima.
                    df_inicial.insert(i, nombre, valores)
                return df_inicial
            # Si no has escogido ninguno de los dos, puestra el mensaje por pantalla
            else:
                print("Debes escoger entre wind o rh")

        # Creamos un método para dejar el dataframe limpio con la media por pais
        def media(self, datafr):
            self.datafr = datafr

            # Eliminamos las columnas en las que estaban las listas de diccionarios, que ya no nos sirven
            datafr.drop(["rh_profile", "wind_profile"], axis= 1, inplace= True)
            # Hacemos un groupby por pais y sacamos la media
            df_media = datafr.groupby("country").mean().reset_index()
            return df_media

        # Creamos un método para unir los dataframes, el de ataques de tiburones y el que hemos creado con la media por país
        def unir(self, df_ataques, df_media_pais):

            df_ultimo = pd.merge(df_ataques, df_media_pais, on= "country", how= "left")
            return df_ultimo

        # Creamos una función para guardar el archivo
        def guardar_csv(self, dt, ubicacion):
            return dt.to_csv(ubicacion)



In [3]:
# Creamos un diccionario con la latitud y longitud de los paises:                         
                        # Pais : [Latitud, Longitud]
paises_coordenadas = {"usa": [39.7837304, -100.445882], 
                        "australia": [-24.7761086, 134.755], 
                        "south africa": [-28.8166236, 24.991639], 
                        "new zealand": [-41.5000831, 172.8344077], 
                        "papua new guinea": [-5.6816069, 144.2489081]}

In [4]:
# Iniciamos la clase
api = Extraccion(paises_coordenadas)

In [5]:
api

In [6]:
# Creamos un dataframe con la llamada a la API
df_meteo = api.llamada_API("meteo")
df_meteo.head(2)

La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion de se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,usa,39.78373,-100.445882
1,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,usa,39.78373,-100.445882


In [7]:
# Creamos un df con los diccionarios de la columna rh_profile separados por columnas
df_rh = api.separar_lista_df(df_meteo, "rh_profile")
df_rh.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 5}","{'layer': '900mb', 'rh': 7}","{'layer': '850mb', 'rh': 8}","{'layer': '800mb', 'rh': 13}","{'layer': '750mb', 'rh': 11}","{'layer': '700mb', 'rh': 7}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 1}","{'layer': '550mb', 'rh': 3}","{'layer': '500mb', 'rh': 10}","{'layer': '450mb', 'rh': 11}","{'layer': '400mb', 'rh': 11}","{'layer': '350mb', 'rh': 13}","{'layer': '300mb', 'rh': 15}","{'layer': '250mb', 'rh': 15}","{'layer': '200mb', 'rh': 14}"


In [8]:
# Creamos un df con los diccionarios de la columna wind_profile separados por columnas
df_wind = api.separar_lista_df(df_meteo, "wind_profile")
df_wind.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 150, 'speed': 2}","{'layer': '900mb', 'direction': 240, 'speed': 2}","{'layer': '850mb', 'direction': 260, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 3}","{'layer': '750mb', 'direction': 280, 'speed': 4}","{'layer': '700mb', 'direction': 265, 'speed': 5}","{'layer': '650mb', 'direction': 270, 'speed': 5}","{'layer': '600mb', 'direction': 265, 'speed': 6}","{'layer': '550mb', 'direction': 255, 'speed': 6}","{'layer': '500mb', 'direction': 250, 'speed': 6}","{'layer': '450mb', 'direction': 245, 'speed': 6}","{'layer': '400mb', 'direction': 250, 'speed': 6}","{'layer': '350mb', 'direction': 250, 'speed': 6}","{'layer': '300mb', 'direction': 250, 'speed': 7}","{'layer': '250mb', 'direction': 245, 'speed': 7}","{'layer': '200mb', 'direction': 250, 'speed': 7}"


In [9]:
# Desempaquetamos el diccionario de df_rh
df_meteo2 = api.desempaquetar(df_rh, df_meteo)

In [10]:
df_meteo2.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,5,7,8,13,11,7,3,1,3,10,11,11,13,15,15,14,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,usa,39.78373,-100.445882
1,6,7,6,8,11,6,3,8,15,16,16,16,16,12,14,6,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,usa,39.78373,-100.445882


In [11]:
# Desempaquetamos el df wind, para la key direction
df_meteo3 = api.desempaquetar(df_wind, df_meteo)

In [12]:
df_meteo3.head(2)

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,150,240,260,275,280,265,270,265,255,250,245,250,250,250,245,250,5,7,8,13,11,7,3,1,3,10,11,11,13,15,15,14,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,usa,39.78373,-100.445882
1,265,330,340,340,325,305,295,275,265,260,250,255,255,245,250,255,6,7,6,8,11,6,3,8,15,16,16,16,16,12,14,6,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,usa,39.78373,-100.445882


In [13]:
# Desempaquetamos el df wind, para la key speed
df_meteo4 = api.desempaquetar(df_wind, df_meteo)

In [14]:
df_meteo4.head(2)

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country,latitud,longitud
0,2,2,3,3,4,5,5,6,6,6,6,6,6,7,7,7,150,240,260,275,280,265,270,265,255,250,245,250,250,250,245,250,5,7,8,13,11,7,3,1,3,10,11,11,13,15,15,14,3,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 150, 'speed':...",16,6,4,1024,none,0,0,110,2,usa,39.78373,-100.445882
1,2,3,4,3,3,3,4,6,6,6,7,7,7,7,7,7,265,330,340,340,325,305,295,275,265,260,250,255,255,245,250,255,6,7,6,8,11,6,3,8,15,16,16,16,16,12,14,6,6,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 265, 'speed':...",15,6,6,1025,none,0,0,220,2,usa,39.78373,-100.445882


In [15]:
# Creamos el df con la media por país
df_meteo_media = api.media(df_meteo4)
df_meteo_media.head()

/tmp/ipykernel_727/4186142874.py:96: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_media = datafr.groupby("country").mean().reset_index()


,country,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud,longitud
0,australia,3.562500,3.578125,3.546875,3.515625,3.421875,3.500000,3.515625,3.546875,3.656250,3.593750,3.593750,3.546875,3.562500,3.921875,4.734375,5.203125,79.921875,80.546875,101.093750,102.109375,103.828125,99.609375,106.406250,96.796875,88.906250,80.468750,82.968750,89.921875,121.953125,129.921875,139.296875,142.109375,13.890625,11.437500,8.484375,6.375000,5.296875,4.468750,3.796875,3.40625,3.078125,3.375000,3.375000,3.203125,2.453125,1.390625,3.562500,5.843750,97.5,2.640625,-9999.0,-9999.0,-9999.0,25.875000,-3.828125,10.828125,1015.937500,2.796875,0.0,3.296875,-24.776109,134.755000
1,new zealand,3.343750,3.234375,3.171875,3.156250,3.187500,3.187500,3.281250,3.421875,3.546875,3.796875,4.031250,4.234375,4.656250,5.015625,6.265625,7.218750,141.875000,143.437500,147.421875,155.390625,179.609375,190.703125,192.890625,212.421875,230.625000,241.796875,255.546875,269.218750,264.921875,255.156250,251.953125,260.312500,13.171875,12.546875,4.625000,2.578125,2.484375,2.812500,3.062500,3.00000,2.718750,2.828125,2.734375,3.875000,5.656250,7.093750,4.984375,1.640625,97.5,6.093750,-9999.0,-9999.0,-9999.0,14.953125,10.265625,10.296875,1020.906250,2.671875,0.0,3.015625,-41.500083,172.834408
2,papua new guinea,4.015625,4.390625,4.453125,4.296875,3.578125,2.765625,2.312500,2.859375,3.437500,4.109375,4.625000,4.750000,4.828125,4.734375,5.312500,6.015625,81.015625,81.484375,82.734375,82.968750,80.000000,70.390625,196.484375,249.375000,255.234375,257.109375,255.625000,249.921875,245.312500,231.953125,206.953125,220.703125,13.437500,12.062500,9.375000,5.562500,2.125000,-0.265625,0.000000,0.40625,0.078125,0.531250,2.593750,3.640625,5.078125,4.968750,4.968750,3.015625,97.5,4.203125,-9999.0,-9999.0,-9999.0,25.796875,-0.718750,11.218750,1009.953125,2.171875,0.0,3.328125,-5.681607,144.248908
3,south africa,2.500000,2.359375,2.234375,2.156250,2.187500,2.140625,2.375000,2.484375,2.734375,2.984375,3.546875,3.703125,4.015625,4.140625,4.531250,4.843750,150.078125,191.250000,182.656250,179.296875,204.296875,222.031250,224.609375,220.703125,228.125000,230.781250,232.812500,237.187500,233.828125,230.078125,223.437500,216.250000,12.921875,9.718750,8.015625,8.609375,9.437500,9.812500,10.203125,8.18750,5.796875,4.781250,3.312500,2.125000,2.984375,3.796875,5.218750,7.765625,97.5,6.015625,-9999.0,-9999.0,-9999.0,23.875000,2.015625,10.218750,1018.859375,1.078125,0.0,2.515625,-28.816624,24.991639
4,usa,2.875000,3.265625,3.750000,4.156250,4.328125,4.421875,4.609375,5.000000,5.281250,5.625000,6.015625,6.375000,6.890625,7.687500,8.250000,8.406250,234.687500,265.546875,301.328125,313.203125,287.265625,305.625000,291.328125,294.453125,293.125000,293.437500,292.968750,288.437500,292.968750,284.375000,274.140625,304.687500,4.375000,5.140625,5.656250,4.718750,3.187500,2.593750,2.296875,2.18750,2.937500,4.000000,4.578125,4.921875,5.312500,6.140625,7.296875,5.984375,97.5,4.593750,-9999.0,-9999.0,-9999.0,12.812500,12.328125,4.281250,1019.562500,0.484375,0.0,2.671875,39.783730,-100.445882


Para unir con los datos de los ataques cargamos el archivo y filtramos el dataframe por la lista de paises de df_meteo_media

In [16]:
df = pd.read_csv("../data/05-tiburon_4.csv", index_col = 0)
lista_paises = df_meteo_media["country"].unique().tolist()
df_ataques = df[df["country"].isin(lista_paises)]
df_ataques.head(2)

,type,country,activity,species,month,fatal,gender,year,age
0,Boating,usa,Paddling,White,Jun,N,F,2018.0,57.0
2,Unprovoked,usa,Walking,Bull,May,N,M,2018.0,15.0


In [17]:
# Unimos en un nuevo df los df de los ataques y de la media del clima por pais
df_ataques_clima = api.unir(df_ataques, df_meteo_media)
df_ataques_clima.head(2)

,type,country,activity,species,month,fatal,gender,year,age,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud,longitud
0,Boating,usa,Paddling,White,Jun,N,F,2018.0,57.0,2.875,3.265625,3.75,4.15625,4.328125,4.421875,4.609375,5.0,5.28125,5.625,6.015625,6.375,6.890625,7.6875,8.25,8.40625,234.6875,265.546875,301.328125,313.203125,287.265625,305.625,291.328125,294.453125,293.125,293.4375,292.96875,288.4375,292.96875,284.375,274.140625,304.6875,4.375,5.140625,5.65625,4.71875,3.1875,2.59375,2.296875,2.1875,2.9375,4.0,4.578125,4.921875,5.3125,6.140625,7.296875,5.984375,97.5,4.59375,-9999.0,-9999.0,-9999.0,12.8125,12.328125,4.28125,1019.5625,0.484375,0.0,2.671875,39.78373,-100.445882
1,Unprovoked,usa,Walking,Bull,May,N,M,2018.0,15.0,2.875,3.265625,3.75,4.15625,4.328125,4.421875,4.609375,5.0,5.28125,5.625,6.015625,6.375,6.890625,7.6875,8.25,8.40625,234.6875,265.546875,301.328125,313.203125,287.265625,305.625,291.328125,294.453125,293.125,293.4375,292.96875,288.4375,292.96875,284.375,274.140625,304.6875,4.375,5.140625,5.65625,4.71875,3.1875,2.59375,2.296875,2.1875,2.9375,4.0,4.578125,4.921875,5.3125,6.140625,7.296875,5.984375,97.5,4.59375,-9999.0,-9999.0,-9999.0,12.8125,12.328125,4.28125,1019.5625,0.484375,0.0,2.671875,39.78373,-100.445882


In [18]:
# Guardamos
api.guardar_csv(df_ataques_clima, "../data/01-ataques-clima.csv")